In [1]:
import yfinance as yf
import pandas as pd
import numpy as np

from tqdm import tqdm
tqdm.pandas()

/Users/dabereabasse/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
start_date = pd.to_datetime("2010-01-01") 
end_date = pd.to_datetime("2020-01-01") # exclusive
interval = '1d'

sp500 = yf.download('^GSPC', interval=interval, start=start_date - pd.Timedelta(days=365), end=end_date)  # S&P 500
gold = yf.download('GC=F', interval=interval, start=start_date - pd.Timedelta(days=365), end=end_date)  # Gold
vix = yf.download('^VIX', interval=interval, start=start_date - pd.Timedelta(days=365), end=end_date)  # VIX
bond_yields = yf.download('^IRX', interval=interval, start=start_date - pd.Timedelta(days=365), end=end_date)  # 13-week Treasury Bill

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
sp500.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,902.989990,934.729980,899.349976,931.799988,931.799988,4048270000
2009-01-05,929.169983,936.630005,919.530029,927.450012,927.450012,5413910000
2009-01-06,931.169983,943.849976,927.280029,934.700012,934.700012,5392620000
2009-01-07,927.450012,927.450012,902.369995,906.650024,906.650024,4704940000
2009-01-08,905.729980,910.000000,896.809998,909.729980,909.729980,4991550000


In [4]:
gold.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,881.500000,881.500000,868.900024,878.799988,878.799988,46
2009-01-05,882.000000,883.500000,847.000000,857.200012,857.200012,35
2009-01-06,855.099976,867.599976,840.000000,865.400024,865.400024,113
2009-01-07,862.000000,867.000000,837.700012,841.099976,841.099976,101
2009-01-08,837.900024,861.000000,837.900024,853.900024,853.900024,255


In [5]:
vix.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,39.580002,39.820000,36.880001,39.189999,39.189999,0
2009-01-05,39.240002,40.220001,38.299999,39.080002,39.080002,0
2009-01-06,38.060001,39.330002,37.340000,38.560001,38.560001,0
2009-01-07,40.290001,43.820000,40.119999,43.389999,43.389999,0
2009-01-08,43.380001,44.599998,42.560001,42.560001,42.560001,0


In [6]:
bond_yields.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,0.095,0.105,0.080,0.085,0.085,0
2009-01-05,0.085,0.105,0.065,0.085,0.085,0
2009-01-06,0.125,0.150,0.070,0.130,0.130,0
2009-01-07,0.115,0.130,0.100,0.100,0.100,0
2009-01-08,0.080,0.100,0.050,0.080,0.080,0


In [7]:
sp500.reset_index(inplace=True)
sp500['sp500_return(%)'] = sp500['Close'].pct_change() * 100

sp500 = sp500[['Date', 'sp500_return(%)']]
sp500.head()

,Date,sp500_return(%)
0,2009-01-02,NaN
1,2009-01-05,-0.466836
2,2009-01-06,0.781713
3,2009-01-07,-3.000962
4,2009-01-08,0.339707


In [8]:
gold.reset_index(inplace=True)
gold['gold_return(%)'] = gold['Close'].pct_change() * 100

gold = gold[['Date', 'gold_return(%)']]
gold.head()

,Date,gold_return(%)
0,2009-01-02,NaN
1,2009-01-05,-2.457894
2,2009-01-06,0.956604
3,2009-01-07,-2.807956
4,2009-01-08,1.521823


In [9]:
vix.reset_index(inplace=True)
vix = vix[['Date', 'Close']]
vix.columns = ['Date', 'vix_close']

vix.head()

,Date,vix_close
0,2009-01-02,39.189999
1,2009-01-05,39.080002
2,2009-01-06,38.560001
3,2009-01-07,43.389999
4,2009-01-08,42.560001


In [10]:
bond_yields.reset_index(inplace=True)
bond_yields = bond_yields[['Date', 'Close']]
bond_yields.columns = ['Date', 'bond_yields_close']

bond_yields.head()

,Date,bond_yields_close
0,2009-01-02,0.085
1,2009-01-05,0.085
2,2009-01-06,0.130
3,2009-01-07,0.100
4,2009-01-08,0.080


In [11]:
# combine all data
data = pd.merge(sp500, gold, on='Date', how='inner')
data = pd.merge(data, vix, on='Date', how='inner')
data = pd.merge(data, bond_yields, on='Date', how='inner')

data.head()

,Date,sp500_return(%),gold_return(%),vix_close,bond_yields_close
0,2009-01-02,NaN,NaN,39.189999,0.085
1,2009-01-05,-0.466836,-2.457894,39.080002,0.085
2,2009-01-06,0.781713,0.956604,38.560001,0.130
3,2009-01-07,-3.000962,-2.807956,43.389999,0.100
4,2009-01-08,0.339707,1.521823,42.560001,0.080


In [12]:
# keep only Date between start_date and end_date
data = data[(data['Date'] >= start_date) & (data['Date'] < end_date)]
data.reset_index(drop=True, inplace=True)

data.head()

,Date,sp500_return(%),gold_return(%),vix_close,bond_yields_close
0,2010-01-04,1.604342,2.054419,20.040001,0.055
1,2010-01-05,0.311568,0.035790,19.350000,0.060
2,2010-01-06,0.054552,1.591991,19.160000,0.045
3,2010-01-07,0.400120,-0.246505,19.059999,0.045
4,2010-01-08,0.288173,0.450091,18.129999,0.040


In [13]:
data.tail()

,Date,sp500_return(%),gold_return(%),vix_close,bond_yields_close
2507,2019-12-24,-0.019545,1.119729,12.67,1.538
2508,2019-12-26,0.512817,0.680413,12.65,1.535
2509,2019-12-27,0.003398,0.298151,13.43,1.518
2510,2019-12-30,-0.578082,0.046238,14.82,1.490
2511,2019-12-31,0.294602,0.330142,13.78,1.505


In [14]:
# save data
data.to_csv('../data/macro_data/macro_data.csv', index=False)

In [8]:
# from wbdata import get_dataframe
# import datetime

# # Sélectionner des indicateurs (par exemple, PIB, taux de chômage)
# indicators = {'NY.GDP.MKTP.CD': 'GDP', 'SL.UEM.TOTL.ZS': 'Unemployment', 'SP.POP.TOTL': 'Population'}

# # Télécharger les données de 2020 à 2023 pour les États-Unis
# data = get_dataframe(indicators, country='USA', date=(datetime.datetime(2010, 1, 1), datetime.datetime(2023, 12, 31)))
# print(data.head())